In [1]:
import feedparser

from bs4 import BeautifulSoup

from whoosh.index import create_in
from whoosh.fields import *
from whoosh.qparser import QueryParser, FuzzyTermPlugin
from whoosh import highlight

import calendar
from datetime import datetime

In [2]:
import feedparser

NewsFeed = feedparser.parse("https://thegeneralist.substack.com/feed")

entry = NewsFeed.entries[0]
print(entry)

{'title': "Revisiting The Generalist's 2020 Predictions", 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://thegeneralist.substack.com/feed', 'value': "Revisiting The Generalist's 2020 Predictions"}, 'summary': 'Welcome to new subscribers. The Generalist helps you master the future by analyzing the most significant tech and financial market trends. Thank you for being here. There are now over 19,000 of us! Hey friends, The IPO frenzy reached its zenith with DoorDash and Airbnb making their public debuts this week. Both got off to an explosive start, with DoorDash finishing the week at a $55.5 billion valuation and Airbnb landing around $83 billion.', 'summary_detail': {'type': 'text/html', 'language': None, 'base': 'https://thegeneralist.substack.com/feed', 'value': 'Welcome to new subscribers. The Generalist helps you master the future by analyzing the most significant tech and financial market trends. Thank you for being here. There are now over 19,000 of us! 

In [3]:
print(entry['title'])
print(entry['author'])
print(entry['summary'])
print(entry['link'])
print(entry['published_parsed'])
content = entry['content'][0]['value']
print(content)

Revisiting The Generalist's 2020 Predictions
MDA Gabriele
Welcome to new subscribers. The Generalist helps you master the future by analyzing the most significant tech and financial market trends. Thank you for being here. There are now over 19,000 of us! Hey friends, The IPO frenzy reached its zenith with DoorDash and Airbnb making their public debuts this week. Both got off to an explosive start, with DoorDash finishing the week at a $55.5 billion valuation and Airbnb landing around $83 billion.
https://thegeneralist.substack.com/p/revisiting-the-generalists-2020-predictions
time.struct_time(tm_year=2020, tm_mon=12, tm_mday=13, tm_hour=17, tm_min=31, tm_sec=0, tm_wday=6, tm_yday=348, tm_isdst=0)
<p></p><p><em>Welcome to new subscribers. The Generalist helps you master the future by analyzing the most significant tech and financial market trends. Thank you for being here. There are now over 19,000 of us!</em></p><div class="captioned-image-container"><figure><a class="image-link image

In [4]:
# Turn HTML into clean text for nice searching

from bs4 import BeautifulSoup
tree = BeautifulSoup(content)

# This validates and cleans HTML but Substack HTML is already fine
# pretty = tree.prettify()
# print(pretty)

pure_text = tree.get_text("\n")
print(pure_text)

Welcome to new subscribers. The Generalist helps you master the future by analyzing the most significant tech and financial market trends. Thank you for being here. There are now over 19,000 of us!
Hey friends,
The IPO frenzy reached its zenith with DoorDash and Airbnb making their public debuts this week. Both got off to an explosive start, with DoorDash finishing the week at a $55.5 billion valuation and Airbnb landing around $83 billion.
If you haven't had a chance yet, you can learn more about DoorDash and Airbnb, below. We also released a report on Affirm, the "buy now, pay later" business, which was expected to IPO in the next couple of weeks. They've delayed their filing along with gaming platform Roblox.
Here's a selection of recent pieces:
​
Affirm in 1 minute
​
​
Airbnb in 1 minute
​
​
DoorDash in 1 minute
​
​
RFS 100, Edition 4
Don't worry, though — our S-1 reports and events will be back with a vengeance as soon as $AFRM and $RBLX reschedule their listings. Stay tuned!
Shar

In [5]:
# Next, build a Whoosh database and search engine
# https://whoosh.readthedocs.io/en/latest/quickstart.html#a-quick-introduction

# Define schema (pretty simple)
# Add documents from all top Substacks
# Build a searcher
# Show results
# Highlight bits from the thinkpieces that match
# Optional: add stemming for better searching
# Later, add support for other newsletters with an RSS feed
# Maybe, later, offer `more_like_this` so people can rabbit-hole in

# Use case: I want to find all articles from my favorite writers that include the term
# "BNPL" since I want to research "Buy Now, Pay Later" in fintech.
# Or I want to find all thinkpieces that mentioned Google in the last year

In [6]:
# Create a list of all RSSes to search

# First, Substacks. Getting their RSS feeds is pretty simple/systematic.
substack_domains = [
    "thegeneralist",
    "danco",
    "diff",
    "nbt",
    "platformer",
    "notboring",
    "sariazout",
    "digitalnative",
    "jamesonstartups",
    "breakingsmart",
    "artofgig",
    "theskip",
    "gwern"
]

# Feeds are, e.g., https://thegeneralist.substack.com/feed
substack_feeds = ["https://{0}.substack.com/feed".format(domain) for domain in substack_domains]

# Now add some custom RSS feeds
# Medium feeds are medium.com/feed/@user or medium.com/feed/publication
custom_feeds = [
    "https://stratechery.com/feed/",
    "https://www.profgalloway.com/feed",
    "https://eugene-wei.squarespace.com/blog?format=rss",
    "https://medium.com/feed/@superwuster",
    "https://commoncog.com/blog/rss",
    "https://www.lennyrachitsky.com/feed",
    "https://medium.com/feed/bloated-mvp",
    "https://daringfireball.net/feeds/main",
    "https://wongmjane.com/api/feed/rss",
    "https://fourweekmba.com/feed",
]

# Unite all feeds into one
all_feeds = substack_feeds + custom_feeds
all_feeds

# We'll read in the feeds from each of these. Get RSS feed at 
# https://thegeneralist.substack.com/feed

# NewsFeed = feedparser.parse("https://thegeneralist.substack.com/feed")

['https://thegeneralist.substack.com/feed',
 'https://danco.substack.com/feed',
 'https://diff.substack.com/feed',
 'https://nbt.substack.com/feed',
 'https://platformer.substack.com/feed',
 'https://notboring.substack.com/feed',
 'https://sariazout.substack.com/feed',
 'https://digitalnative.substack.com/feed',
 'https://jamesonstartups.substack.com/feed',
 'https://breakingsmart.substack.com/feed',
 'https://artofgig.substack.com/feed',
 'https://theskip.substack.com/feed',
 'https://gwern.substack.com/feed',
 'https://stratechery.com/feed/',
 'https://www.profgalloway.com/feed',
 'https://eugene-wei.squarespace.com/blog?format=rss',
 'https://medium.com/feed/@superwuster',
 'https://commoncog.com/blog/rss',
 'https://www.lennyrachitsky.com/feed',
 'https://medium.com/feed/bloated-mvp',
 'https://daringfireball.net/feeds/main',
 'https://wongmjane.com/api/feed/rss',
 'https://fourweekmba.com/feed']

In [7]:
# Convenience function to safely get an item from a dict.
# If the key doesn't exist, just returns none
def safe_get(obj, key):
    if obj.has_key(key):
        return obj[key]
    else:
        return None

In [8]:
# Allows optional case-sensitive searches. all-lowercase is case insensitive, 
# any capital letters makes it case sensitive 

class CaseSensitivizer(analysis.Filter):
    def __call__(self, tokens):
        for t in tokens:
            yield t
            if t.mode == "index":
               low = t.text.lower()
               if low != t.text:
                   t.text = low
                   yield t

ana = analysis.RegexTokenizer() | CaseSensitivizer()
# [t.text for t in ana("The new SuperTurbo 5000", mode="index")]
# ["The", "the", "new", "SuperTurbo", "superturbo", "5000"]



In [17]:
# Build the database
schema = Schema(
    title=TEXT(stored=True),
    author=TEXT(stored=True),
    publication=TEXT(stored=True),
    summary=TEXT(stored=True),
    url=TEXT(stored=True),
    published=DATETIME(stored=True, sortable=True),
    content=TEXT(stored=True, analyzer=ana))
index = create_in("whoosh_index2", schema)
writer = index.writer()

# Read every item from our RSS feeds into there
# Call writer.add_document() repeatedly for each item 

for feed_url in all_feeds:
    print(feed_url)
    news_feed = feedparser.parse(feed_url)
    
    # NOTE: we can only get the last few entries from this RSS feed.
    # Substack doesn't seem to show anything older than the last 20.
    # So we should build in a system to start caching these.
    
#     print(len(news_feed.entries))

    for entry in news_feed.entries:
        
        # Get publication name. This is in the feed's `feed` field, along with other metadata
        publication = None
        metadata = safe_get(news_feed, 'feed')
        if metadata is not None:
            publication = safe_get(metadata, 'title')

        # Clean up the date into a normal datetime
        clean_datetime = datetime.fromtimestamp(calendar.timegm(entry['published_parsed']))
        
        # Most feeds put the main content in `content`,
        # but a rare few like Eugene Wei put it in `summary`
        # (in which case `content` is empty). With this logic, let's get a single `content` field.
        body_text = None
        # See if `content` exists
        content_holder = safe_get(entry, 'content')
        if content_holder is not None:
            # We have content; fill it in
            content_tree = BeautifulSoup(content_holder[0]['value'])
            body_text = content_tree.get_text(" ", strip=True)
        else:
            # No content provided. `summary` must hold all the text.
            summary_tree = BeautifulSoup(safe_get(entry, 'summary'))
            body_text = summary_tree.get_text(" ", strip=True)

        writer.add_document(
            title=safe_get(entry, 'title'),
            author=safe_get(entry, 'author'),
            publication=publication,
            summary=safe_get(entry, 'summary'),
            url=safe_get(entry, 'link'),
            published=clean_datetime,
            content=body_text)

print("DONE!")
writer.commit()

https://thegeneralist.substack.com/feed
https://danco.substack.com/feed
https://diff.substack.com/feed
https://nbt.substack.com/feed
https://platformer.substack.com/feed
https://notboring.substack.com/feed
https://sariazout.substack.com/feed
https://digitalnative.substack.com/feed
https://jamesonstartups.substack.com/feed
https://breakingsmart.substack.com/feed
https://artofgig.substack.com/feed
https://theskip.substack.com/feed
https://gwern.substack.com/feed
https://stratechery.com/feed/
https://www.profgalloway.com/feed
https://eugene-wei.squarespace.com/blog?format=rss
https://medium.com/feed/@superwuster
https://commoncog.com/blog/rss
https://www.lennyrachitsky.com/feed
https://medium.com/feed/bloated-mvp
https://daringfireball.net/feeds/main
https://wongmjane.com/api/feed/rss
https://fourweekmba.com/feed
DONE!


In [10]:
# For convenience, we're overriding the standard fragment formatter
class BracketFormatter(highlight.Formatter):
    """Puts square brackets around the matched terms.
    """

    def format_token(self, text, token, replace=False):
        # Use the get_text function to get the text corresponding to the
        # token
        tokentext = highlight.get_text(text, token, replace)

        # Return the text as you want it to appear in the highlighted
        # string
        return "[[%s]]" % tokentext

In [21]:
# Try searching
from whoosh.qparser import QueryParser, MultifieldParser
import whoosh.qparser as qparser

search_term = "Notion"

with index.searcher() as searcher:
    parser = QueryParser("content", index.schema)
    # Allow fuzzy matching (EDIT: kinda screws things up)
    # parser.add_plugin(FuzzyTermPlugin())
    # Allow searching for entire phrases w/ single quotes, like 'microsoft teams'
    parser.add_plugin(qparser.SingleQuotePlugin())
    
    query = parser.parse(search_term)
    results = searcher.search(query, limit=None)
    
    # Highlighting settings
    # This provides more context characters around the searched-for text
    results.fragmenter.surround = 50
    results.fragmenter.maxchars = 500
    
    # Surround matched tags with brackets
    results.formatter = BracketFormatter()
    
    # Convert each Hit into a dict
    def extract_hit_info(hit):
        return {
            'title': hit.get('title'),
            'publication': hit.get('publication'),
            'author': hit.get('author'),
            'url': hit.get('url'),
            'highlights': hit.highlights("content", top=3),
            'published': hit.get('published'),
            'score': hit.score
        }
    
    hit_list = [extract_hit_info(h) for h in results]
    
    print(hit_list)

[{'title': 'Roam Research MVP Review: Pages + Bullets = Cool New Product Category', 'publication': 'Bloated MVP - Medium', 'author': 'Liron Shapira', 'url': 'https://medium.com/bloated-mvp/roam-research-mvp-review-pages-bullets-cool-new-product-category-28baf5dd0a3?source=rss----d5a38f5a82b---4', 'highlights': 'that popular tools like Evernote, Google Docs and [[Notion]] are flawed for research and note-taking because...And if he had switched to another tool like [[Notion]] or Workflowy instead of Roam, there...to offer. So compared to Evernote, Google Docs, [[Notion]] and Workflowy, is Roam ultimately', 'published': datetime.datetime(2020, 1, 15, 15, 50, 3), 'score': 6.746579207210092}, {'title': 'Summer Vacation', 'publication': "Alex Danco's Newsletter", 'author': 'Alex Danco', 'url': 'https://danco.substack.com/p/summer-vacation', 'highlights': 'stuff in the replies too, and Will Haynes made a [[Notion]] page to keep track of essays mentioned in the replies', 'published': datetime.

In [12]:
# Experimental

news_feed = feedparser.parse("https://diff.substack.com/feed")
print(news_feed['feed'])

{'title': 'The Diff', 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://diff.substack.com/feed', 'value': 'The Diff'}, 'subtitle': 'Inflections in finance and tech', 'subtitle_detail': {'type': 'text/html', 'language': None, 'base': 'https://diff.substack.com/feed', 'value': 'Inflections in finance and tech'}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://diff.substack.com'}, {'href': 'https://diff.substack.com/feed', 'rel': 'self', 'type': 'application/rss+xml'}], 'link': 'https://diff.substack.com', 'image': {'href': 'https://cdn.substack.com/image/fetch/w_256,c_limit,f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fbucketeer-e05bbc84-baa3-437e-9518-adb32be77984.s3.amazonaws.com%2Fpublic%2Fimages%2F5b730ae4-aa35-496e-8198-965a187e2e43_600x600.png', 'title': 'The Diff', 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://diff.substack.com/feed', 'value': 'The Diff'}, 'links': [{'rel': 'alternate', 'type': 'text/

In [13]:
# TODO: read from the thinkpiecer module and use it here, instead of writing custom code

In [14]:
z = {'a':5, 'b':3}

In [20]:
print(z.get('c'))

None


In [27]:
# Search for the newest items.

# Try searching
from whoosh.qparser import QueryParser, MultifieldParser
import whoosh.qparser as qparser
from whoosh.qparser.dateparse import DateParserPlugin

search_term = "'-26 weeks to now'"

with index.searcher() as searcher:
    parser = QueryParser("published", index.schema)
    # Allow fuzzy matching (EDIT: kinda screws things up)
    # parser.add_plugin(FuzzyTermPlugin())
    # Allow searching for entire phrases w/ single quotes, like 'microsoft teams'
    parser.add_plugin(qparser.SingleQuotePlugin())
    
    # Add the DateParserPlugin to the parser
    parser.add_plugin(DateParserPlugin())
    
    query = parser.parse(search_term)
    results = searcher.search(query, limit=50, sortedby="published", reverse=True)
    
    # Highlighting settings
    # This provides more context characters around the searched-for text
#     results.fragmenter.surround = 50
#     results.fragmenter.maxchars = 500
    
    # Surround matched tags with brackets
#     results.formatter = BracketFormatter()
    
    # Convert each Hit into a dict
    def extract_hit_info(hit):
        return {
            'title': hit.get('title'),
            'publication': hit.get('publication'),
            'author': hit.get('author'),
            'url': hit.get('url'),
#             'highlights': hit.highlights("content", top=3),
            'published': hit.get('published'),
            'score': hit.score
        }
    
    hit_list = [extract_hit_info(h) for h in results]
    
    print(hit_list)

[{'title': "Why labels on misinformation aren't working", 'publication': 'Platformer', 'author': 'Casey Newton', 'url': 'https://www.platformer.news/p/why-labels-on-misinformation-arent', 'published': datetime.datetime(2020, 12, 14, 20, 0, 1), 'score': 63743572801000000}, {'title': 'Raymond Wong Reviews Fitness+', 'publication': 'Daring Fireball', 'author': 'John Gruber', 'url': 'https://www.inputmag.com/reviews/apple-fitness-plus-got-me-hooked-on-working-out', 'published': datetime.datetime(2020, 12, 14, 19, 38, 54), 'score': 63743571534000000}, {'title': 'Todd Haselton Reviews Apple Fitness+', 'publication': 'Daring Fireball', 'author': 'John Gruber', 'url': 'https://www.cnbc.com/2020/12/14/apple-fitness-plus-review-release-date-price.html', 'published': datetime.datetime(2020, 12, 14, 19, 21, 54), 'score': 63743570514000000}, {'title': '‘Listen Up, Georgia’', 'publication': 'Daring Fireball', 'author': 'John Gruber', 'url': 'https://twitter.com/adamlisagor/status/1338602407933034496

In [1]:
from thinkpiecer import *

# Let's try the real code
# Load the real index and search it 

ix = load_index()

print(get_recent_articles(ix))

FileIndex(FileStorage('./whoosh_prod'), 'MAIN')
5
[{'title': "Why labels on misinformation aren't working", 'publication': 'Platformer', 'author': 'Casey Newton', 'url': 'https://www.platformer.news/p/why-labels-on-misinformation-arent', 'published': datetime.datetime(2020, 12, 14, 20, 0, 1), 'score': 63743572801000000}, {'title': 'EHR interoperability: why it matters', 'publication': 'In Silico', 'author': 'Scott Xiao', 'url': 'https://insilico.substack.com/p/ehr-interoperability-why-it-matters', 'published': datetime.datetime(2020, 12, 14, 17, 44, 23), 'score': 63743564663000000}, {'title': '"personal social networking" is the working market definition, in the legal jargon.', 'publication': 'Stories by Tim Wu on Medium', 'author': 'Tim Wu', 'url': 'https://medium.com/@superwuster/personal-social-networking-is-the-working-market-definition-in-the-legal-jargon-663c4544aa25?source=rss-7d8da5c1a768------2', 'published': datetime.datetime(2020, 12, 14, 16, 20, 28), 'score': 63743559628000